# Expected Value of Cost function in population setting
## Projecting $p$ entries as sigmoids 
We have 

$$C(A, P) = \text{Tr}\left((B^* - B)\Sigma_X(B^* - B)\right),$$

where $$B = P^{-1}AP.$$

Here, $A$ is lower triangular and $P$ is a doubly stochastic matrix, namely

\begin{align*}
0 \leq &p_{ij} &\forall\ i,j = 1, \cdots, n\\
\sum_{j = 1}^n &p_{ij} = 1&\forall\ i = 1, \cdots, n\\
\sum_{i=1}^n &p_{ij} = 1 &\forall\ j = 1, \cdots, n\\
\end{align*}

## Dealing with the constraints
### Non-negativity constraints
To deal with the constraints that $p$ must be non-negative, we project the entries by applying the sigmoid function of each entry. Note that the range of the sigmoid is $(0, 1)$, which is also what we require for our entries $p$ (we did not explicitly constrain $p$ to be smaller than 1, but the equality constraints combined with the non-negativity constraints make sure that is the case.

Hence, 

$$P = \sigma(P_{\sigma}),$$

where $\sigma(\cdot)$ indicates the element-wise application of the sigmoid function 

$$\sigma(x) = \frac{1}{1 + e^{-x}}.$$

So, instead of directly estimating $P$, we estimate $P_\sigma$. The constraints now becomes an equality constrained program

\begin{align*}
\sum_{j = 1}^n &p_{ij} = 1&\forall\ i = 1, \cdots, n\\
\sum_{i=1}^n &p_{ij} = 1 &\forall\ j = 1, \cdots, n\\
\end{align*}

## Dealing with equality constraints
To deal with the $2n$ equality constraints, we use $\textit{lagrange multipliers}$ $\lambda_{row, i}$, $i = 1, \cdots, n$ and $\lambda_{col, j}$, $j = 1, \cdots, n$. The dual function now becomes 

$$\mathcal{L}(A, P_\sigma, \mathbf{\lambda}) = C(A, P) - \sum_{i=1}^n \lambda_{row, i} \left(\sum_{j = 1}^n p_{ij} - 1\right) - \sum_{j = 1}^n \lambda_{col, j} \left(\sum_{i = 1}^n p_{ij} - 1\right),$$
where $P = \sigma(P_\sigma)$

## Optimizing $\mathcal{L}(A, P_\sigma, \mathbf{\lambda})$
let $$q(\mathbf{\lambda}) = \inf_{A, P_{sigma}} \mathcal{L}(A, P_\sigma, \mathbf{\lambda}).$$
Then the $\textit{dual problem}$ is $$\max_{\mathbf{\lambda}} q(\mathbf{\lambda}).$$

## Deriving $q(\mathbf{\lambda})$
To minimize $\mathcal{L}(A, P_\sigma, \mathbf{\lambda})$, we will use gradient descent.

The partial derivative with respect to entries in $A$ is simple, 

$$\frac{\partial\mathcal{L}(A, P_\sigma, \mathbf{\lambda})}{\partial a_{ij}} = \frac{\partial C(A, P)}{\partial a_{ij}}$$

Now, the partial derivative with respect to the entries in $p$ is more involved as it occurs also in exactly two equality constraints, namely in $\lambda_{row, i}$ and in $\lambda{col, j}$.

$$\frac{\partial\mathcal{L}(A, P_\sigma, \mathbf{\lambda})}{\partial p_{ij}} = \frac{\partial C(A, P)}{\partial p_{ij}} - \lambda_{row, i}p_{ij} - \lambda_{col, j}p_{ij}.$$

Now, as $C(A, P)$ is unfortunately yet inevitably non-convex, we cannot expect to find the infimum $q(\mathbf{\lambda})$. However, let us consider a local minimum $\tilde{q}(\mathbf{\lambda})$, which we will find by gradient descent. Let us start with an initial matrix $A_0$ (e.g. the non-zero matrix) and an initial unconstrained matrix $P_{0,\sigma} = Z$, where $Z \sim \mathcal{N}(\mathbf{0}, I_{n^2})$. So, $P_\sigma$ is close to zero, but there is some perturbation to make sure the matrix is non-singular.

For a sufficiently small step size, we will do this gradient descent until we have found a stationary point:
$$(A_{t + 1}, P_{t + 1}) = (A_t, P_t) - \eta\ \nabla \mathcal{L}(A_t, P_t, \mathbf{\lambda})$$
Note that this is an $\textit{unconstrained}$ optimization problem now, which should make things more easy. 

Suppose that after a number of iterations, we have found this local minimum with parameters $\tilde{A}$, $\tilde{P}$. Then, we have that $$\tilde{q}(\mathbf{\lambda}) = \mathcal{L}(\tilde{A}, \tilde{P}, \mathbf{\lambda}).$$

## Deriving the dual problem
Now that we have $\tilde{q}(\mathbf{\lambda})$, the solution to the dual problem is
$$\max_{\lambda} \tilde{q}(\mathbf{\lambda}),$$
which is again an unconstrained optimization problem, which we can also optimize by maximizing $\lambda$'s.

In [1]:
import numpy as np

In [2]:
# actual solution
n = 3
As = np.tril(np.random.rand(n, n))
Ps = np.identity(n)
Sigma = np.identity(n)

print(f"A*:\n{np.round(As, 2)}\n\nP*:\n{Ps}")

A*:
[[0.3  0.   0.  ]
 [0.23 0.89 0.  ]
 [0.85 0.9  0.29]]

P*:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [3]:
def expected_cost(A, P, As = As, Ps = Ps):
    # base on the distribution of X, no actual data needed.
    # we need the covariance of X_t - X_{t-1}.
    # Then, the expected cost is the trace of this covariance
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))
    
    covariance_matrix = Sigma + np.matmul((Bs - B), np.matmul(covariance_X, (Bs - B).transpose()))
    
    return np.trace(covariance_matrix)

In [4]:
print(f"Expected cost: {expected_cost(np.random.rand(n,n), Ps)}.")

Expected cost: 12.88024614190274.


In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# get gradient of our outcome
def grad_a(A, Psigma, i, j, As = As, Ps = Ps):
    P = sigmoid(Psigma)
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    J = np.zeros((n, n))
    J[i][j] = 1
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))

    return -2 * np.trace(np.matmul(covariance_X, np.matmul((Bs - B).transpose(), np.matmul(P_inv, np.matmul(J, P)))))

def grad_p(A, Psigma, i, j, As = As, Ps = Ps):
    P = sigmoid(Psigma)
    P_inv = np.linalg.inv(P)
    Ps_inv = np.linalg.inv(Ps)
    
    B = np.matmul(P_inv, np.matmul(A, P))
    Bs = np.matmul(Ps_inv, np.matmul(As, Ps))
    
    J = np.zeros((n, n))
    J[i][j] = 1
    
    covariance_X = np.matmul(np.linalg.inv(np.identity(n ** 2) - np.kron(Bs, Bs)), Sigma.reshape(n ** 2)).reshape((n, n))

    B_grad = np.matmul(P_inv, np.matmul(A, J))
    B_grad -= np.matmul(P_inv, np.matmul(J, np.matmul(P_inv, np.matmul(A, P))))
    
    return -2 * np.trace(np.matmul(covariance_X, np.matmul((Bs - B).transpose(), B_grad))) * sigmoid(P[i][j]) * (1 - sigmoid(P[i][j]))

def get_gradient(A, P):
    gradient = np.zeros(int(n * (n + 1) / 2 + n ** 2))
    index = 0
    
    for i in range(n):
        for j in range(i + 1):
            gradient[index] = grad_a(A, P, i, j)
            index += 1
            
    for i in range(n):
        for j in range(n):
            gradient[index] = grad_p(A, P, i, j)
            index += 1
    return gradient

0.19661193324148185
[[ 0.          0.09685158  0.        ]
 [-0.29737716  0.         -0.85045194]
 [ 0.          0.40104964  0.        ]]
-271.2496227567504


In [9]:
A = np.zeros((n, n))
P = np.ones((n, n)) / n + np.random.rand(n, n) / n
print(P)
eta = 1e-6

[[0.4083684  0.39949239 0.33551572]
 [0.55767729 0.51830324 0.66418083]
 [0.58863107 0.61089676 0.41700066]]


In [14]:
eta = 1e-4

In [16]:
grad = get_gradient(A, P)
print(np.sum(grad))
while np.sum(np.abs(grad)) > 8:
    A_grad = np.zeros((n , n))
    A_grad[np.tril_indices(n)] = grad[:int(n * (n + 1) / 2)]
    P_grad = grad[int(n * (n + 1) / 2):].reshape((n , n))

    A -= eta * A_grad * 1
    P -= eta * P_grad
    
    grad = get_gradient(A, P)
    print(expected_cost(A, sigmoid(P)))    
    
# P = sinkhorn_balance(P)

# print(A_grad)
# print(P_grad)
#print(A, P)
#print(expected_cost(A, P))

-8.714501214120054
4.69127839812642
4.689678020209154
4.6880809089565
4.686487057420481
4.684896458668038
4.6833091057809435
4.681724991855852
4.680144110004187
4.678566453352152
4.676992015040706
4.67542078822549
4.673852766076839
4.672287941779723
4.6707263085337125
4.669167859552983
4.667612588066237
4.666060487316695
4.6645115505620725
4.66296577107453
4.661423142140645
4.6598836570614
4.6583473091521235
4.65681409174247
4.655283998176407
4.6537570218121465
4.652233156022151
4.650712394193082
4.649194729725773
4.647680156035196
4.646168666550431
4.6446602547146565
4.643154913985085
4.641652637832951
4.640153419743481
4.638657253215857
4.637164131763198
4.635674048912509
4.634186998204672
4.632702973194406
4.63122196745023
4.629743974554457
4.628268988103123
4.626797001706011
4.625328008986575
4.623862003581927
4.622398979142826
4.620938929333609
4.619481847832196
4.6180277283300395
4.616576564532117
4.615128350156871
4.613683078936215
4.61224074461547
4.61080134095336
4.60936486172

In [17]:
print(A,P)

[[0.65369742 0.         0.        ]
 [0.38003372 0.3548804  0.        ]
 [0.16511378 0.25673041 0.25471694]] [[0.71642165 0.18074869 0.25404891]
 [0.47405659 0.55458431 0.69625908]
 [0.47998471 0.77333756 0.34366889]]


In [5791]:
def sinkhorn_balance(P):
    for _ in range(10000):
        # normalize rows
        r_sum = P.sum(axis=1)
        P = P / r_sum[:, np.newaxis]
        
        
        c_sum = P.sum(axis = 0)
        P = P / c_sum
    return P

Ps = np.array([[0.4, 0.1, 0.5], [0.1, 0.4, 0.6], [1, 2, 4]])
print(sinkhorn_balance(Ps))

[[0.59965663 0.1187942  0.28154917]
 [0.15568219 0.4934595  0.35085831]
 [0.24466118 0.3877463  0.36759252]]
